In [3]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import GroupedLayerControl
from folium.plugins import MousePosition
from folium.plugins import Fullscreen
from folium.plugins import MarkerCluster
from shapely import geometry

In [4]:
data = gpd.read_file('C:/Users/user/OneDrive/Рабочий стол/GIS_PRO/pr_3_2/Vyborg_border.geojson') #Базовый слой от которого строится карта (тут границы города)
data_food_pnt = gpd.read_file('C:/Users/user/OneDrive/Рабочий стол/GIS_PRO/pr_3_2/Vyborg_food_points.geojson') # Слой с точками заведений общепита (кафе и рестораны)
data_route = gpd.read_file('C:/Users/user/OneDrive/Рабочий стол/GIS_PRO/pr_3_2/Vyborg_foot_route_line.geojson') # Слой с пешеходными туристическими маршрутами
data_grid = gpd.read_file('C:/Users/user/OneDrive/Рабочий стол/GIS_PRO/pr_3_2/Vyborg_places_grid.geojson') # Слой с сеткой для интерполяции
data_places = gpd.read_file('C:/Users/user/OneDrive/Рабочий стол/GIS_PRO/pr_3_2/Vyborg_places_points.geojson') # Слой с точками мест туристического интереса (сувениры, достопримечательности, отели)
data_water_route = gpd.read_file('C:/Users/user/OneDrive/Рабочий стол/GIS_PRO/pr_3_2/Vyborg_water_route_line.geojson') # Слой с водными туристическими маршрутами
data_places_poly = gpd.read_file('C:/Users/user/OneDrive/Рабочий стол/GIS_PRO/pr_3_2/Vyborg_places_polig.geojson') # Слой с полигонами основных достопримечательностей

# Переводим в WGS84
data_food_pnt = data_food_pnt.to_crs('EPSG:4326')
data_route = data_route.to_crs('EPSG:4326')
data_grid = data_grid.to_crs('EPSG:4326')
data_places = data_places.to_crs('EPSG:4326')
data_water_route = data_water_route.to_crs('EPSG:4326')
data_places_poly = data_places_poly.to_crs('EPSG:4326')

In [5]:
# Создание карты по координатам слоя

center_lat = data.centroid.y.mean()
center_lon = data.centroid.x.mean()
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    control_scale=True,
    tiles=None
)

# MapTiler слой (нашли очень красивый акварельный, как раз для туристической карты)
aquarelle_layer = folium.TileLayer(
    tiles="https://api.maptiler.com/maps/aquarelle/{z}/{x}/{y}.png?key=urot3wPYoEBliJRLcn0d",
    attr='<a href="https://www.maptiler.com/copyright/" target="_blank">© MapTiler</a> <a href="https://www.openstreetmap.org/copyright" target="_blank">© OpenStreetMap contributors</a>',
    name="MapTiler Aquarelle",
    overlay=False,
    control=False,
    show=True
).add_to(m)

# Решили добавить несколько дополнительных подложек
# Обычные OSM-улицы
osm_layer = folium.TileLayer(
    tiles="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
    attr='© OpenStreetMap',
    name="Улицы (OSM)",
    overlay=False,
    control=False,
    show=False
).add_to(m)

# Спутниковый слой
satellite_layer = folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Google Satellite",
    name="Google Спутник",
    overlay=False,
    control=False,
    show=False
).add_to(m)

C:\Users\user\AppData\Local\Temp\ipykernel_17384\1440012753.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = data.centroid.y.mean()
C:\Users\user\AppData\Local\Temp\ipykernel_17384\1440012753.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = data.centroid.x.mean()


In [6]:
# Добавляем контент!

# Достопримечательности в виде полигонов (показывают свое название при наведении и нажатии)
visit_points = folium.GeoJson(
    data_places_poly,
    name="Достопримечательности",
    tooltip=folium.GeoJsonTooltip(
        fields=["name"],
        aliases=[""],
        style=("font-family: Arial; font-size: 14px; padding: 5px;")
    ),
    popup=folium.GeoJsonPopup(
        fields=['name'],
        aliases=[""],
        style="font-size: 16px; font-weight: bold;",
        max_width=250
    ),
    style_function=lambda x: {
        "fillColor": "#C3CAEA", # свои цвета
        "color": "#808EBF", # свои цвета
        "weight": 2,
        "fillOpacity": 0.5
    },
    highlight_function=lambda x: {
        "fillColor": "#F6CDC9",
        "fillOpacity": 0.8,
        "weight": 3
    },
    zoom_on_click=True,
    show=False
).add_to(m)

#--------------------------------------------------------------------------------------------
# Пешеходные маршруты (два вида, без дополнительной ифнормации)
route1p_layer = folium.FeatureGroup(name='Короткий пеший маршрут', show=False)
route2p_layer = folium.FeatureGroup(name='Длинный пеший маршрут', show=False)

def add_route_to_layer(gdf, layer, route_id, color, weight=4):
    route = gdf[gdf['type'] == route_id]
    folium.GeoJson(
        route,
        style_function=lambda x: {
            'color': color,
            'weight': weight,
            'opacity': 0.8
        },
    ).add_to(layer)

add_route_to_layer(data_route, route1p_layer, route_id=1, color='#D25E2F')  # Красный
add_route_to_layer(data_route, route2p_layer, route_id=2, color='#545FA2')  # Синий

route1p_layer.add_to(m)
route2p_layer.add_to(m)

#--------------------------------------------------------------------------------------------
# Водные маршруты (5 штук) с дополнительной информацией о цене и времени маршрута
route1_layer = folium.FeatureGroup(name='Бирюзовое кольцо Выборга', show=False)
route2_layer = folium.FeatureGroup(name='Легенды Монрепо', show=False)
route3_layer = folium.FeatureGroup(name='Старый шлюз Лавола', show=False)
route4_layer = folium.FeatureGroup(name='Древняя Тура', show=False)
route5_layer = folium.FeatureGroup(name='Экспресс', show=False)

def add_route_to_layer(gdf, layer, route_id, color, weight=4):
    route = gdf[gdf['type'] == route_id]
    folium.GeoJson(
        route,
        style_function=lambda x: {
            'color': color,
            'weight': weight,
            'opacity': 0.8
        },
        popup=folium.GeoJsonPopup(
            fields=['description'],
            aliases=['Описание маршрута:'],
            localize=True,
            sticky=True
        )
    ).add_to(layer) 

add_route_to_layer(data_water_route, route1_layer, route_id=1, color='#00CED1')  # Бирюзовый
add_route_to_layer(data_water_route, route2_layer, route_id=2, color='#32CD32')  # Лайм
add_route_to_layer(data_water_route, route3_layer, route_id=3, color='#FFA500')  # Ораньжевый
add_route_to_layer(data_water_route, route4_layer, route_id=4, color='#800080')  # Фиолетовый
add_route_to_layer(data_water_route, route5_layer, route_id=5, color='#FF0000')  # Красный

route1_layer.add_to(m)
route2_layer.add_to(m)
route3_layer.add_to(m)
route4_layer.add_to(m)
route5_layer.add_to(m)

#--------------------------------------------------------------------------------------------
# Работа с сеткой и интерполяцией (всё было сделано заранее, поэтому только стиль настраиваем)
grid = folium.Choropleth(
    geo_data=data_grid,
    data=data_grid,
    columns=['id', 'one_sum'],
    fill_color= "Blues", # Цветовая палитра
    line_opacity=0.2,
    key_on='feature.properties.id',
    legend_name="Количество объектов",
    name='Плотность достопримечательностей',
    highlight=True,
    bins=4,
    show=False
).add_to(m)

#--------------------------------------------------------------------------------------------
# Точки общепита (2 вида - кафе и рестораны - у каждого своя иконка + информация о названии и адрес)
gdf_cafe = data_food_pnt
gdf_cafe = gdf_cafe[gdf_cafe.geometry.notnull() & (gdf_cafe.geometry.type == "Point")]
if gdf_cafe.crs != "EPSG:4326":
    gdf_cafe = gdf_cafe.to_crs("EPSG:4326")

cafe_layer = folium.FeatureGroup(name="Кафе", show=True)
cafe_cluster = MarkerCluster(
    options={"maxClusterRadius": 60}
).add_to(cafe_layer)

type_settings = {
    1: {'color': 'blue', 'icon': 'coffee'},
    2: {'color': 'red', 'icon': 'cutlery'}
}

# Дополнительная информация о названии и адресе
for _, row in gdf_cafe.iterrows():
    lat, lon = row.geometry.y, row.geometry.x
    cafe_type = row.get("type", 0)
    name = row.get("name", "Без названия")
    address = row.get("adress", "")
    settings = type_settings.get(cafe_type, {'color': 'gray', 'icon': 'info-sign'})

    popup = folium.Popup(f"""
        <div style="line-height:1.2">
            <div style="font-weight:bold; font-size:14px;">{name}</div>
            <div style="font-size:11px; color:gray;">{address}</div>
        </div>
    """, max_width=250)

    folium.Marker(
        location=[lat, lon],
        popup=popup,
        icon=folium.Icon(color=settings['color'], icon=settings['icon'], prefix='fa')
    ).add_to(cafe_cluster)

cafe_layer.add_to(m)

#--------------------------------------------------------------------------------------------
# Точки притяжения туристов в городе (3 вида - достопримечательности, сувениры и отели + вокзал
gdf_places = data_places
gdf_places = gdf_places[gdf_places.geometry.notnull() & (gdf_places.geometry.type == "Point")]
if gdf_places.crs != "EPSG:4326":
    gdf_places = gdf_places.to_crs("EPSG:4326")

places_layer = folium.FeatureGroup(name="Места в городе", show=True)
places_cluster = MarkerCluster(
    options={"maxClusterRadius": 60}
).add_to(places_layer)

type_settings = {
    1: {'color': 'orange', 'icon': 'star'},     # достопримечательности
    2: {'color': 'purple', 'icon': 'shopping-bag'},        # сувениры
    3: {'color': 'green', 'icon': 'bed'},        # отели
    4: {'color': 'black', 'icon': 'train'},        # вокзал
}

# Дополнительная информация при нажатии: название и кол-во звезд для Отелей
for _, row in gdf_places.iterrows():
    lat, lon = row.geometry.y, row.geometry.x
    places_type = row.get("type", 0)
    name = row.get("name", "Без названия")
    description = row.get("description", "") or ""
    settings = type_settings.get(places_type, {'color': 'gray', 'icon': 'info-sign'})
    
    popup = folium.Popup(f"""
        <div style="line-height:1.2">
            <div style="font-weight:bold; font-size:14px;">{name}</div>
            <div style="font-size:11px; color:gray;">{description}</div>
        </div>
    """, max_width=250)

    folium.Marker(
        location=[lat, lon],
        popup=popup,
        icon=folium.Icon(color=settings['color'], icon=settings['icon'], prefix='fa')
    ).add_to(places_cluster)

places_layer.add_to(m)

In [7]:
# GroupedLayerControl (управление слоями). Делим на два для подложек и всего остального.
# Подложку можно выбирать только одну, а остальные слои можно включать одновременно
GroupedLayerControl(
    groups={
        "Подложки": [aquarelle_layer, osm_layer, satellite_layer],
    },
    exclusive_groups=True,
    collapsed=True
).add_to(m)

GroupedLayerControl(
    groups={
        "Пешеходные маршруты": [route1p_layer, route2p_layer],
        "Водные маршруты": [route1_layer, route2_layer, route3_layer, route4_layer, route5_layer],
        "Места на карте": [grid, places_layer, cafe_layer, visit_points]
    },
    exclusive_groups=False,
    collapsed=True
).add_to(m)

# Дополнительные виджеты (MousePosition и Fullscreen)
MousePosition().add_to(m)
Fullscreen(
    position="topleft",
    title="Fullscreen",
    title_cancel="Window mode",
    force_separate_button=True,
).add_to(m)

# Мини карта (хз зачем)
from folium.plugins import MiniMap

MiniMap(
    tile_layer= 'cartodb positron',
    position='bottomleft',
    width=150,
    height=150
).add_to(m)

# Поиск (тоже не зна зачем, но прикольно)
from folium.plugins import Geocoder

Geocoder(
    position='topleft',
    collapsed=True,  # Сворачиваемый вид
    provider='nominatim',  # Использует OpenStreetMap
    placeholder='Поиск места...'
).add_to(m)

# Отображаем карту
m

In [8]:
m.save("C:/Users/user/OneDrive/Рабочий стол/GIS_PRO/pr_3_2/index.html")